In [1]:
import pandas as pd
import numpy as np
from itertools import product

In [49]:
df=pd.read_csv('population_hhld.csv')
#process Year
df['Year'] = df['Year'].str[1:].astype(int)
#create datapoint_available column
df['datapoint_available']=df['Value'].notna().astype(int)

#replace 'Not applicable' with Total  to apply groupby later on, and get rid of NAs
columns_to_replace = ['Age group', 'Area', 'Nationality']

# Replace "Not applicable" with "total" only in those columns
df[columns_to_replace] = df[columns_to_replace].replace('Not applicable', 'Total')
df['Sex'] = df['Sex'].replace('Not applicable', 'Both sexes')

df = df.dropna(subset=['Value'])

df.head()

,Indicator,Country,Age group,Area,Sex,Nationality,Year,Value,datapoint_available
0,Occupied housing units by average household si...,Iraq,Total,Total,Both sexes,Nationals,2021,6.4,1
1,Occupied housing units by average household si...,Iraq,Total,Total,Both sexes,Non-nationals,2021,5.6,1
2,Occupied housing units by average household si...,Kuwait,Total,Total,Both sexes,Nationals,2005,8.6,1
3,Occupied housing units by average household si...,Kuwait,Total,Total,Both sexes,Non-nationals,2005,5.6,1
4,Occupied housing units by average household si...,Kuwait,Total,Total,Both sexes,Nationals,2011,5.7,1


#### dataframe to show indicators, countries, sex and nationality disaggregation labels

In [50]:
# Assuming your DataFrame is named df
df_aggregation_uniquevalues = df.groupby(['Indicator']).agg({
    'Area': lambda x: list(x.dropna().unique()),
    'Sex': lambda x: list(x.dropna().unique()),
    'Age group': lambda x: list(x.dropna().unique())
}).reset_index()



In [51]:
df_aggregation_uniquevalues.to_excel('aggregation_uniquevalues.xlsx', index=False)

In [52]:
df_aggregation_uniquevalues

,Indicator,Area,Sex,Age group
0,Age specific fertility rates,[Total],[Both sexes],"[[15-19], [20-24], [25-29], [30-34], [35-39], ..."
1,Average annual population growth rate (%),[Total],[Both sexes],[Total]
2,Children (<5 years) mortality rates,"[Total, Urban, Rural]","[Male, Female, Both sexes]",[Total]
3,Female headed households occupied housing unit...,"[Total, Urban, Rural]",[Both sexes],[Total]
4,Infant (<1) mortality rates,"[Total, Urban, Rural]","[Male, Female, Both sexes]",[Total]
5,International migrant stock,[Total],"[Male, Female, Both sexes]",[Total]
6,Life expectancy at birth for total population,[Total],"[Male, Female, Both sexes]",[Total]
7,Mean age at first marriage,[Total],"[Male, Female, Both sexes]",[Total]
8,Occupied housing units by average household si...,"[Total, Urban, Rural]",[Both sexes],[Total]
9,Population Size,[Total],"[Both sexes, Female, Male]",[Total]


In [53]:
indicator_criteria={
    'Age specific fertility rates':{'aggregate':'Sex','keep':['Both sexes']},
    'Average annual population growth rate (%)':{'aggregate':'Sex','keep':['Both sexes']},
    'Children (<5 years) mortality rates':{'aggregate':'Sex','keep':['Both sexes','Male', 'Female']},
    'Female headed households occupied housing units (%)':{'aggregate':'Area','keep':['Total','Urban', 'Rural']},
    'Infant (<1) mortality rates':{'aggregate':'Sex','keep':['Both sexes','Male', 'Female']},
    'International migrant stock':{'aggregate':'Sex','keep':['Both sexes','Male', 'Female']},
    'Life expectancy at birth for total population':{'aggregate':'Sex','keep':['Both sexes','Male', 'Female']},
    'Mean age at first marriage':{'aggregate':'Sex','keep':['Both sexes','Male', 'Female']},
    'Occupied housing units by average household size (%)':{'aggregate':'Area','keep':['Total','Urban', 'Rural']},
    'Population Size':{'aggregate':'Sex','keep':['Both sexes','Male', 'Female']},
    'Population estimates':{'aggregate':'Sex','keep':['Both sexes','Male', 'Female']},
    'Registered divorces':{'aggregate':'Sex','keep':['Both sexes']},
    'Registered marriages':{'aggregate':'Sex','keep':['Both sexes']},
    'Sex ratio':{'aggregate':'Sex','keep':['Both sexes']},
    'Total fertility rates':{'aggregate':'Sex','keep':['Both sexes']},
    'Total number of refugees':{'aggregate':'Sex','keep':['Both sexes']}
}

In [54]:
def availability(group,agg_col):
    '''check in each group if the unique values for column Area or Sex are subset of T,M,F or T,U,R and set availability accordingly'''

    # Initialize availability
    group['availability'] = 0
    vals_to_check1={}
    vals_to_check2={}

    #get the unique aggregate levels and make it a set
    levels=set(group[agg_col].unique())
    #get the values to check against as being subset or not
    if agg_col=='Sex':
        vals_to_check1={'Both sexes','Male', 'Female'}
        vals_to_check2={'Male', 'Female'}
    if agg_col=='Area':
        vals_to_check1={'Total','Urban', 'Rural'}
        vals_to_check2={'Urban', 'Rural'}

    #availability= 1 for total response, 0 for partial response
    if vals_to_check1.issubset(levels) or vals_to_check2.issubset(levels):
        availability=1
    else:
        availability=0

    group['availability']=availability

    return group

In [ ]:
'''loop though indicators. only keep the relevant columns, for example if its on sex disaggregated we dont need the nationality
orelse keeping it the condition in the above function def availability(group,cnt) wont work since we will end up with count>cnt'''

indicators=list(df['Indicator'].unique())

df_list=[]

for ind in indicators:
    #filter on the indicator
    df_sub=df[df['Indicator']==ind].copy()
    criteria=indicator_criteria[ind]

    #get the column and value to filter on
    agg_col=criteria['aggregate']
    to_keep=criteria['keep']
    

    df_filtered=df_sub[df_sub[agg_col].isin(to_keep)].copy()
    #keep the relevant columns only
    df_filtered1=df_filtered[['Indicator','Country', 'Year',agg_col,'Value']]
    #groupby by col to remove any repetitions coming from other disaggregations
    df_filtered2 = df_filtered1.groupby(['Indicator','Country', 'Year',agg_col], as_index=False).agg({'Value': 'first'})
    #group by sex and calculate total/partial availability, the group_keys=False not to include group labels as part of the index
    df_grouped = df_filtered2.groupby(['Indicator','Country', 'Year'], group_keys=False).apply(availability,agg_col)
    #append to the df_list
    df_list.append(df_grouped)


final_df = pd.concat(df_list, ignore_index=True)
final_df.to_excel('grouped.xlsx',index=False)


In [56]:
final_df['Indicator'].unique()

array(['Occupied housing units by average household size (%)',
       'Female headed households occupied housing units (%)',
       'Registered marriages', 'Registered divorces',
       'Mean age at first marriage', 'Population estimates',
       'Average annual population growth rate (%)', 'Sex ratio',
       'Total fertility rates', 'Age specific fertility rates',
       'Life expectancy at birth for total population',
       'Total number of refugees', 'Infant (<1) mortality rates',
       'Children (<5 years) mortality rates', 'Population Size',
       'International migrant stock'], dtype=object)

#### get the response rates

In [57]:
df_summary =final_df.groupby(['Indicator', 'Country', 'Year']).agg({'availability': 'first'}).reset_index()
df_summary.to_excel('availability_summary.xlsx', index=False)

In [58]:

def calculate_availability_percentage(group):
    # Count availability values (0s and 1s)
    counts = group['availability'].value_counts()

    # Calculate percentages
    percentages = (counts / 25) * 100

    # Map percentages back to each row
    group['availability_percentage'] = group['availability'].map(percentages)

    return group


availability_rr = df_summary.groupby(['Indicator', 'Country'], group_keys=False).apply(calculate_availability_percentage)

#groupby on the availability column (use () to format it multiline)
availability_rr_grouped=(availability_rr.groupby(['Indicator', 'Country','availability'])
                         .agg({'availability_percentage': 'max'})
                         .reset_index()
)

availability_rr_grouped.to_excel('availability_RR.xlsx', index=False)

availability_rr_grouped.head()

C:\Users\511232\AppData\Local\Temp\ipykernel_15796\1321475979.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  availability_rr = df_summary.groupby(['Indicator', 'Country'], group_keys=False).apply(calculate_availability_percentage)


,Indicator,Country,availability,availability_percentage
0,Age specific fertility rates,Algeria,0,44.0
1,Age specific fertility rates,Bahrain,0,88.0
2,Age specific fertility rates,Comoros,0,4.0
3,Age specific fertility rates,Egypt,0,88.0
4,Age specific fertility rates,Iraq,0,88.0


#### create a table having all the years 2000 to 2025

In [59]:
#Create full list of years
years = list(range(2000, 2026))

#Get unique indicators and countries
indicators = availability_rr['Indicator'].unique()
countries = availability_rr['Country'].unique()

#Create the cartesian product of all (Indicator, Country, Year)
full_index = pd.DataFrame(list(product(indicators, countries, years)), columns=['Indicator', 'Country', 'Year'])

#Merge with the original data
availability_final = pd.merge(full_index, availability_rr, on=['Indicator', 'Country', 'Year'], how='left')


#add the colorcode column
def color_map(val):
    if pd.isna(val):
        return "#D3D3D3"  # Grey for missing
    elif val == 1:
        return "#A9CBA3"  # Green for full response
    elif val == 0:
        return "#A5C8E1"  # Blue for partial response
    else:
        return "#FFFFFF"  # Fallback/white

availability_final['ColorCode'] = availability_final['availability'].apply(color_map)



availability_final.to_excel('availability_final.xlsx', index=False)
